## Modelling

### Install Pycaret

In [ ]:
# if on Colab, install pycaret and other dependencies
%pip install --pre pycaret
%pip install xgboost
%pip install shap
%pip install explainerdashboard
%pip install deepchecks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.8/287.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.24.0
    Un

### Mount Google Collab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Imports

In [ ]:
import numpy as np
import pandas as pd

### Load Dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/cleaned_data.csv")
df.head()

,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
0,19.14,0.394962,0.883161,71745.401048,5.814362e+07,0
1,18.50,0.530341,1.185878,109949.757148,5.580105e+07,1
2,21.45,0.136319,0.304818,24865.506798,6.720689e+07,0
3,20.63,0.198863,0.444672,78890.076805,3.039644e+07,0
4,22.70,0.076658,0.171412,56036.519484,6.311863e+07,0


# Setup a new experiment

The only required parameters are 'data' and 'target'.

However, it is a good option to name your experiment. You can also specify a session_id, if you didn't by default a random seed is generated and returned in the Information grid. The unique number is then distributed as a seed in all functions used during the experiment. This can be used for later reproducibility of the entire experiment.

In [ ]:
# init setup
from pycaret.classification import *
exp_name = setup(data = df,  target = 'is_hazardous',
                 session_id=123,
                 log_experiment=False,
                 experiment_name='exp_1',
                 use_gpu=False)

,Description,Value
0,Session id,123
1,Target,is_hazardous
2,Target type,Binary
3,Original data shape,"(338171, 6)"
4,Transformed data shape,"(338171, 6)"
5,Transformed train set shape,"(236719, 6)"
6,Transformed test set shape,"(101452, 6)"
7,Numeric features,5
8,Preprocess,True
9,Imputation type,simple


### Automatically compare models
Get the list of models in library and their id that can be used in functions.


In [ ]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


This function train all the models available in the model library and scores them using cross-validation. The output of this function is a scoring grid with average cross-validated scores.

In [ ]:
# compare models
best = compare_models(exclude='catboost') # catboost will take too long to train.

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9166,0.9472,0.5891,0.7084,0.6432,0.5965,0.5997,19.0590
rf,Random Forest Classifier,0.9139,0.9452,0.5630,0.7032,0.6253,0.5773,0.5819,53.7390
xgboost,Extreme Gradient Boosting,0.8918,0.9132,0.2648,0.7017,0.3845,0.3382,0.3870,2.0720
lightgbm,Light Gradient Boosting Machine,0.8886,0.9067,0.2066,0.7216,0.3212,0.2803,0.3467,9.8660
gbc,Gradient Boosting Classifier,0.8856,0.8956,0.1600,0.7412,0.2631,0.2282,0.3095,57.3640
dt,Decision Tree Classifier,0.8850,0.7451,0.5572,0.5490,0.5530,0.4870,0.4871,2.0230
ada,Ada Boost Classifier,0.8838,0.8856,0.1537,0.7070,0.2525,0.2168,0.2933,12.2030
dummy,Dummy Classifier,0.8724,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1270
ridge,Ridge Classifier,0.8723,0.8347,0.0000,0.0333,0.0001,-0.0001,-0.0014,0.1460
qda,Quadratic Discriminant Analysis,0.8721,0.8512,0.0042,0.4420,0.0083,0.0058,0.0287,0.1730


Processing:   0%|          | 0/33 [00:00<?, ?it/s]

### Manually train most promising models with custom parameters

Train again some of the best models. The accuracy obtained by default will be the same obtained with the `compare_models` function with the default config. However, in this case it is possible to provide custom hyperparameters to improve the model.  

In [ ]:
# train et
et = create_model('et',
                   fold = 10,
                   return_train_score = True) # this allows checking overfitting

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(et)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False)


In [ ]:
# train rf
rf = create_model('rf',
                   fold = 10,
                   n_estimators = 100,
                   max_depth=6,
                   return_train_score = True) # this allows checking overfitting

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(rf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)


In [ ]:
# train xgboost
xgb = create_model('xgboost',
                   fold = 10,
                   max_depth=4,
                   return_train_score = True) # this allows checking overfitting

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(xgb)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=-1,
              num_parallel_tree=None, objective='binary:logistic', ...)


### Automatically optimize models

This function tunes the hyperparameters of the model. The output of this function is a scoring grid with cross-validated scores by fold. The best model is selected based on the metric defined in optimize parameter.

In [ ]:
tuned_et = tune_model(et, return_train_score = True, n_iter = 100) # n_iter is the number of models to be tested

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10.3 µs


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
# compare the hyperparameters
print(et) # default model
print(tuned_et) # tuned model

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False)
ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, ve